<a href="https://colab.research.google.com/github/Catofood/PixelMind/blob/Linear-and-NN-structure/Linear_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

# ---------- Функции активации ----------
def ReLU(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

# ---------- Функции потерь ----------
def categorical_cross_entropy_loss(y_pred: np.array, y_true: np.array) -> float:
    eps = 1e-9
    return - np.mean(np.sum(y_true * np.log(y_pred + eps), axis=1))

def mse_loss(y_pred: np.array, y_true: np.array) -> float:
    return np.mean((y_pred - y_true)**2)


In [3]:
class Linear:

  def __init__(self, input_size: int, output_size: int, activation_func=None):
    self.n_in = input_size
    self.n_out = output_size
    self.weights = np.random.randn(self.n_in, self.n_out)
    self.bias = np.random.randn(1, self.n_out)

    if activation_func is None:
      self.activation_func = lambda x: x
    else:
      self.activation_func = activation_func


  def forward(self, x: np.array):
     # прямой проход по слою (вычисление значений)
     # y = w * x + b [здесь имеются в виду матричные операции]
     # (x должен быть двумерным: [batch_size, input_size])
     linear_res = np.dot(x, self.weights) + self.bias
     return self.activation_func(linear_res)


In [4]:
class NeuralNetwork:
    """
    Нейронная сеть, состоящая из нескольких слоёв (Linear).
    """
    def __init__(self, layers=None, loss_func=None):
        # Инициализируем список слоёв
        if layers is None:
            self.layers = []
        else:
            self.layers = layers

        # Функцию потерь передаём напрямую
        # Если ничего не указали, можно по умолчанию брать MSE
        self.loss_func = loss_func if loss_func is not None else mse_loss

    def add_layer(self, layer: Linear):
        self.layers.append(layer)

    def forward(self, x: np.array):
        """
        Прямой проход через все слои.
        """
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def compute_loss(self, x: np.array, y_true: np.array) -> float:
        """
        Выполняет прямой проход + функцию потерь
        """
        y_pred = self.forward(x)
        return self.loss_func(y_pred, y_true)

In [6]:
if __name__ == "__main__":
    # Допустим, у нас есть задача многоклассовой классификации на 3 класса

    # 1) Создаем сеть:
    #    Первый слой: вход 4 -> выход 5 (ReLU)
    #    Второй слой: вход 5 -> выход 3 (softmax)
    #    Важно: обычно softmax делаем либо как отдельный слой, либо внутри forward последнего слоя.

    layers = [
        Linear(4, 5, ReLU),         # [batch, 4] -> [batch, 5]
        Linear(5, 3, softmax)      # [batch, 5] -> [batch, 3], затем softmax
    ]

    # Укажем, что функция потерь — это наша categorical_cross_entropy_loss
    model = NeuralNetwork(layers=layers, loss_func=categorical_cross_entropy_loss)

    # Пример входных данных (batch_size=2, 4 признака)
    x = np.array([
        [1, 0, 2, 3],
        [2, -1, 0, 1]
    ])
    # Целевые метки (two samples) в one-hot. Предположим, 3 класса:
    # Первый сэмпл (batch[0]) принадлежит классу 2 (индекс 1), второй — классу 3 (индекс 2).
    # В питоне удобнее так: класс i -> y[i-1] = 1
    # Пусть класс считается с 0, тогда:
    #   y_true[0] = [0, 1, 0], y_true[1] = [0, 0, 1].
    y_true = np.array([
        [0, 1, 0],  # класс 1 (считая с 0)
        [0, 0, 1],  # класс 2 (считая с 0)
    ])

    # Считаем функцию потерь
    loss_value = model.compute_loss(x, y_true)
    print(f"Loss (categorical cross-entropy): {loss_value:.6f}")

    # Можно проверить, что модель возвращает (y_pred):
    y_pred = model.forward(x)
    print("y_pred (вероятности по классам):\n", y_pred)
    # Сумма по строкам (batch) должна быть = 1
    print("Сумма по каждой строке:", y_pred.sum(axis=1))

Loss (categorical cross-entropy): 3.724248
y_pred (вероятности по классам):
 [[7.60753171e-01 9.65037661e-04 2.38281791e-01]
 [3.96443730e-01 1.43697231e-04 6.03412572e-01]]
Сумма по каждой строке: [1. 1.]
